### Creación de gpkg de division

In [1]:
from utils_pre_procesamiento import generar_tiles_spatial_split

tif_sierra = r"D:\Shapefiles\PI\CO_2510221339013\VOL_PER1_ORT_001_002705\IMG_PER1_ORT_PMS_002705\IMG_PER1_20210706153514_ORT_PMS_002705.TIF"
out_dir = "./"

generar_tiles_spatial_split(tif_sierra, out_dir = out_dir)

Píxeles válidos totales: 588882000
Val   asignado: 70667800 (12.0%)
Test asignado: 70662900 (12.0%)
Train: 447551300


### Creacion de parches

In [ ]:
from raster_to_tile import data_split

tif_selva = r"D:\Shapefiles\PI\CO_2512022106159\VOL_PER1_ORT_001_010438\IMG_PER1_ORT_PMS_010438\IMG_PER1_20230922151914_ORT_PMS_010438.TIF"
tif_sierra = r"D:\Shapefiles\PI\CO_2510221339013\VOL_PER1_ORT_001_002705\IMG_PER1_ORT_PMS_002705\IMG_PER1_20210706153514_ORT_PMS_002705.TIF"

mask_casas_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\buildings_32718_selva.gpkg"
mask_casas_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\buildings_32717_sierra.gpkg"

mask_caminos_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\capa_carreteras_selva.gpkg"
mask_caminos_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\\capa_carreteras_sierra.gpkg"

gpkg_division_selva_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\division_en_tile_selva.gpkg"
gpkg_division_sierra_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\division_en_tile_sierra.gpkg"

dataset_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSelvaMulticlase"
dataset_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase"

dataset_general_out = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetGeneral"

### Area Excluida
gpkg_area_ciudad = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\area_excluida_sierra.gpkg"

#Sierra
data_split(tif_sierra, mask_casas_sierra_path, mask_caminos_sierra_path, gpkg_division_sierra_tile, dataset_general_out, gpkg_area_excluida_path=gpkg_area_ciudad)

In [ ]:
#Selva
data_split(tif_selva, mask_casas_selva_path, mask_caminos_selva_path, gpkg_division_selva_tile, dataset_general_out)

### Verificar GPU disponible

In [ ]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices('GPU'))


In [ ]:
from utils2 import split_dataset

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset(carpeta=dataset_general_out)

In [ ]:
from TiffDataGeneratorAugmentedUnet import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, normalize='imagenet',batch_size=1, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


In [18]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [ ]:
from tensorflow.keras.optimizers import Adam
import configuraciones as cfg
import tensorflow as tf


BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=3,
    activation='softmax'
    #input_shape=(512, 512, 4)
)

# Definir nombres de clases
class_names = ['background', 'road', 'building']

model.compile(
    optimizer=Adam(1e-4),
    loss=cfg.masked_sparse_cce,
    metrics=[
        cfg.MaskedSparseCategoricalAccuracy(num_classes=3, class_names=class_names, name='acc'),
        cfg.MaskedIoUPerClass(num_classes=3, class_names=class_names, name='miou')
    ]
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_loss',      # métrica que se evalúa
    mode='min',
    patience=10,
    restore_best_weights=True,
    verbose= 1
)

checkpoint = ModelCheckpoint(
    r"checkpoint.keras",         # ruta del archivo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau( # reduccion lr auto
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop, checkpoint, reduce_lr, cfg.PrintMetricsPerClass()]
)
print("Epochs realizadas:", len(history.history['loss']))

In [ ]:
# Después del entrenamiento, obtener IoU por clase
iou_metric = model.metrics[-1]  # Última métrica
iou_per_class = iou_metric.get_iou_per_class()

print("\nIoU por clase:")
for class_name, iou_value in iou_per_class.items():
    print(f"  {class_name}: {iou_value:.4f}")

In [ ]:
# Save final model
model.save("model_final.keras")

# Evaluate on train set
print("\n=== Evaluación en test set ===")
train_results = model.evaluate(train_gen, verbose=1)
print(f"Train Loss: {train_results[0]:.4f}")
print(f"Train Masked Accuracy: {train_results[1]:.4f}")
print(f"Train Masked Mean IoU: {train_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en val set ===")
val_results = model.evaluate(val_gen, verbose=1)
print(f"Val Loss: {val_results[0]:.4f}")
print(f"Val Masked Accuracy: {val_results[1]:.4f}")
print(f"Val Masked Mean IoU: {val_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en test set ===")
test_results = model.evaluate(test_gen, verbose=1)
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Masked Accuracy: {test_results[1]:.4f}")
print(f"Test Masked Mean IoU: {test_results[2]:.4f}")